In [1]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json

In [2]:
ccvi = pd.read_csv('../resources/ccvi_nonulls.csv')
ccvi


,FIPS,stateName,countyName,ccvi,theme1,theme2,theme3,theme4,theme5,theme6,theme7
0,1001020100,ALABAMA,Autauga,0.441972,0.458336,0.377478,0.170811,0.841617,0.966294,0.209805,0.287591
1,1001020200,ALABAMA,Autauga,0.742619,0.562935,0.584587,0.683826,0.768947,0.982399,0.209805,0.427810
2,1001020300,ALABAMA,Autauga,0.778072,0.245094,0.645024,0.472226,0.863762,0.953086,0.724817,0.439041
3,1001020400,ALABAMA,Autauga,0.362727,0.023280,0.370119,0.192050,0.977978,0.894654,0.209805,0.425573
4,1001020500,ALABAMA,Autauga,0.816600,0.319725,0.421440,0.764060,0.753240,0.978307,0.749454,0.497597
...,...,...,...,...,...,...,...,...,...,...,...
72177,56043000200,WYOMING,Washakie,0.193098,0.632400,0.106362,0.792014,0.669875,0.126091,0.259923,0.008196
72178,56043000301,WYOMING,Washakie,0.288511,0.536803,0.358751,0.539916,0.400654,0.199599,0.259923,0.589599
72179,56043000302,WYOMING,Washakie,0.574666,0.804595,0.349800,0.613380,0.701398,0.137185,0.815509,0.273972
72180,56045951100,WYOMING,Weston,0.112481,0.476381,0.238316,0.834326,0.559188,0.149020,0.037122,0.008224


In [3]:
# city = 'Los Angeles'
state = 'California'

states ={ 
    "Alabama": {'num': "01", 'abbr': 'AL'},
    "Alaska": {'num': "02", 'abbr': 'AK'},
    "Arizona": {'num': "04", 'abbr': 'AZ'},
    "Arkansas": {'num': "05", 'abbr': 'AR'},
    "California": {'num': "06", 'abbr': 'CA'},
    "Colorado": {'num': "08", 'abbr': 'CO'},
    "Delaware": {'num': "10", 'abbr': 'CT'},
    "District of Columbia": {'num': "11", 'abbr': 'DE'},
    "Connecticut": {'num': "09", 'abbr': 'DC'},
    "Florida": {'num': "12", 'abbr': 'FL'},
    "Georgia": {'num': "13", 'abbr': 'GA'},
    "Idaho": {'num': "16", 'abbr': 'ID'},
    "Hawaii": {'num': "15", 'abbr': 'HI'},
    "Illinois": {'num': "17", 'abbr': 'IL'},
    "Indiana": {'num': "18", 'abbr': 'IN'},
    "Iowa": {'num': "19", 'abbr': 'IA'},
    "Kansas": {'num': "20", 'abbr': 'KS'},
    "Kentucky": {'num': "21", 'abbr': 'KS'},
    "Louisiana": {'num': "22", 'abbr': 'LA'},
    "Maine": {'num': "23", 'abbr': 'ME'},
    "Maryland": {'num': "24", 'abbr': 'MD'},
    "Massachusetts": {'num': "25", 'abbr': 'MA'},
    "Michigan": {'num': "26", 'abbr': 'MI'},
    "Minnesota": {'num': "27", 'abbr': 'MN'},
    "Mississippi": {'num': "28", 'abbr': 'MS'},
    "Missouri": {'num': "29", 'abbr': 'MO'},
    "Montana": {'num': "30", 'abbr': 'MT'},
    "Nebraska": {'num': "31", 'abbr': 'NE'},
    "Nevada": {'num': "32", 'abbr': 'NV'},
    "New Hampshire": {'num': "33", 'abbr': 'NH'},
    "New Jersey": {'num': "34", 'abbr': 'NJ'},
    "New Mexico": {'num': "35", 'abbr': 'NM'},
    "New York": {'num': "36", 'abbr': 'NY'},
    "North Carolina": {'num': "37", 'abbr': 'NC'},
    "North Dakota": {'num': "38", 'abbr': 'ND'},
    "Oklahoma": {'num': "40", 'abbr': 'OK'},
    "Oregon": {'num': "41", 'abbr': 'OR'},
    "Pennsylvania": {'num': "42", 'abbr': 'PA'},
    "Rhode Island": {'num': "44", 'abbr': 'RI'},
    "South Carolina": {'num': "45", 'abbr': 'SC'},
    "South Dakota": {'num': "46", 'abbr': 'SD'},
    "Tennessee": {'num': "47", 'abbr': 'TN'},
    "Texas": {'num': "48", 'abbr': 'TX'},
    "Vermont": {'num': "50", 'abbr': 'VT'},
    "Utah": {'num': "49", 'abbr': 'UT'},
    "Virginia": {'num': "51", 'abbr': 'VA'},
    "Washington": {'num': "53", 'abbr': 'WA'},
    "West Virginia": {'num': "54", 'abbr': 'WV'},
    "Wisconsin": {'num': "55", 'abbr': 'WI'},
    "Wyoming": {'num': "56", 'abbr': 'WY'},
    "Puerto Rico": {'num': "72", 'abbr': 'PR'}
}

In [4]:
# city = f'{city} city'
state_num = states[state]['num']
# full_city = f'{city}, {state}'
# county_nums = []
# county_query = ''
tracts = []
pops = 'B01003_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B03001_003E'
demogs = {
    'total': {},
    'white': {},
    'black': {},
    'native': {},
    'asian': {},
    'pacific': {},
    'hispanic': {}
}




In [5]:

url = f'https://api.census.gov/data/2019/acs/acs5?get=NAME,{pops}&for=tract:*&in=state:{state_num}'

response = requests.get(url).json()

pd.DataFrame(response)

for r in response:
    if r[0] != 'NAME':
        tracts.append({
            'FIPS': int(f'{r[8]}{r[9]}{r[10]}'),
            'total': int(r[1]),
            'white': int(r[2]),
            'black': int(r[3]),
            'native': int(r[4]),
            'asian': int(r[5]),
            'pacific': int(r[6]),
            'hispanic': int(r[7])
        })



In [6]:
populations = pd.DataFrame(tracts)
populations

ccvi_and_pop = pd.merge(populations, ccvi, on='FIPS')

races = ['total','white','black','native','asian', 'pacific', 'hispanic']

for race in races:
    demogs[race]['population'] = int(ccvi_and_pop[race].sum())
    demogs[race]['ccvi'] = (ccvi_and_pop[race]*ccvi_and_pop['ccvi']).sum()/demogs[race]['population']
    demogs[race]['population_percent'] = (demogs[race]['population']/demogs['total']['population'])*100
    
demogs



{'total': {'population': 39184759,
  'ccvi': 0.6107227243815876,
  'population_percent': 100.0},
 'white': {'population': 23405251,
  'ccvi': 0.5748261136565029,
  'population_percent': 59.73049623707013},
 'black': {'population': 2253401,
  'ccvi': 0.7053331771802169,
  'population_percent': 5.750707819843934},
 'native': {'population': 302002,
  'ccvi': 0.6672118803011934,
  'population_percent': 0.7707129192755785},
 'asian': {'population': 5684633,
  'ccvi': 0.5768182276242306,
  'population_percent': 14.50725523155572},
 'pacific': {'population': 154637,
  'ccvi': 0.6362910543608903,
  'population_percent': 0.39463557757239237},
 'hispanic': {'population': 15288340,
  'ccvi': 0.7351319132562741,
  'population_percent': 39.0160368218674}}

In [7]:
covid = pd.read_csv('../resources/CRDT_Data.csv')
covid

# # for c in covid.columns:
# #     print(c)

covid = covid.loc[covid['State'] == states[state]['abbr'],:]


cases = covid[['Cases_Total','Cases_White','Cases_Black','Cases_AIAN','Cases_Asian','Cases_NHPI','Cases_Latinx']]
deaths = covid[['Deaths_Total','Deaths_White','Deaths_Black','Deaths_AIAN','Deaths_Asian','Deaths_NHPI','Deaths_Latinx']]




In [8]:
for i in range(0, len(cases.columns)):
    demogs[races[i]]['cases'] = int(cases[cases.columns[i]].values[0])
    demogs[races[i]]['percent_of_cases'] = (demogs[races[i]]['cases']/demogs['total']['cases'])*100
    demogs[races[i]]['discrepancy_percent'] = (demogs[races[i]]['percent_of_cases']/demogs[races[i]]['population_percent'])*100
#     demogs[races[i]]['discrepancy_difference'] = demogs[races[i]]['percent_of_cases'] - demogs[races[i]]['population_percent']
#     demogs[races[i]]['infected_percent'] = (demogs[races[i]]['cases']/demogs[races[i]]['population'])*100
    
    demogs[races[i]]['deaths'] = int(deaths[deaths.columns[i]].values[0])
    demogs[races[i]]['percent_of_deaths'] = (demogs[races[i]]['deaths']/demogs['total']['deaths'])*100
    demogs[races[i]]['chance_of_death'] = (demogs[races[i]]['deaths']/demogs[races[i]]['cases'])*100
    demogs[races[i]]['deaths_discrepancy_percent'] = (demogs[races[i]]['percent_of_deaths']/demogs[races[i]]['population_percent'])*100
#     demogs[races[i]]['deaths_discrepancy_difference'] = demogs[races[i]]['percent_of_deaths'] - demogs[races[i]]['population_percent']
#     demogs[races[i]]['died_percent'] = (demogs[races[i]]['deaths']/demogs[races[i]]['population'])*100

demogs


{'total': {'population': 39184759,
  'ccvi': 0.6107227243815876,
  'population_percent': 100.0,
  'cases': 3501394,
  'percent_of_cases': 100.0,
  'discrepancy_percent': 100.0,
  'deaths': 54124,
  'percent_of_deaths': 100.0,
  'deaths_discrepancy_percent': 100.0,
  'chance_of_death': 1.5457843361815322},
 'white': {'population': 23405251,
  'ccvi': 0.5748261136565029,
  'population_percent': 59.73049623707013,
  'cases': 546630,
  'percent_of_cases': 15.611782050234849,
  'discrepancy_percent': 26.137037248563516,
  'deaths': 16586,
  'percent_of_deaths': 30.644446086763725,
  'deaths_discrepancy_percent': 51.30452284396906,
  'chance_of_death': 3.034227905530249},
 'black': {'population': 2253401,
  'ccvi': 0.7053331771802169,
  'population_percent': 5.750707819843934,
  'cases': 111279,
  'percent_of_cases': 3.178134194552227,
  'discrepancy_percent': 55.26509595193582,
  'deaths': 3275,
  'percent_of_deaths': 6.050920109378464,
  'deaths_discrepancy_percent': 105.22044066468807,
  

In [9]:
demographics = pd.DataFrame(demogs).drop(columns=['total'])
demographics




,white,black,native,asian,pacific,hispanic
population,2.340525e+07,2.253401e+06,302002.000000,5.684633e+06,154637.000000,1.528834e+07
ccvi,5.748261e-01,7.053332e-01,0.667212,5.768182e-01,0.636291,7.351319e-01
population_percent,5.973050e+01,5.750708e+00,0.770713,1.450726e+01,0.394636,3.901604e+01
cases,5.466300e+05,1.112790e+05,9025.000000,1.865620e+05,15281.000000,1.509103e+06
percent_of_cases,1.561178e+01,3.178134e+00,0.257754,5.328221e+00,0.436426,4.310006e+01
discrepancy_percent,2.613704e+01,5.526510e+01,33.443644,3.672797e+01,110.589669,1.104676e+02
deaths,1.658600e+04,3.275000e+03,184.000000,6.105000e+03,322.000000,2.440200e+04
percent_of_deaths,3.064445e+01,6.050920e+00,0.339960,1.127965e+01,0.594930,4.508536e+01
deaths_discrepancy_percent,5.130452e+01,1.052204e+02,44.109821,7.775181e+01,150.754315,1.155560e+02
chance_of_death,3.034228e+00,2.943053e+00,2.038781,3.272371e+00,2.107192,1.616987e+00


In [12]:
demographics.loc['ccvi'].values

for row in demographics.index:
    print(type(row))

# index = demographics.index
# a_list = list(index)
# print(a_list)

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [44]:
for_max = {}

# def likelyhood(row):
#     values = demographics.loc[row].values
#     for i in range(1, len(races)):
#         if races[i] != 'total':
#             for_max[races[i]] = {}
#             for_max[races[i]][row] = (values[i-1]/values.sum())*100
# #             check.append((values[i-1]/values.sum())*100)

for row in list(demographics.index):
    
    values = demographics.loc[row].values
        
    for i in range(1, len(races)):
        
        # get population numbers
        if row == 'population':
            for_max[races[i]] = {}
            for_max[races[i]][row] = int(values[i-1])
                
        # calculate ccvi values
        elif row == 'ccvi':
            for_max[races[i]][row] = 100-(values[i-1])*100
        
        # calculate chances for where next infection will occure
        elif row == 'discrepancy_percent':
            for_max[races[i]]['chance_of_infection'] = (values[i-1]/values.sum())*100
            
        elif row == 'chance_of_death':
            for_max[races[i]][row] = values[i-1]
            
    
    

            
            
    
            
    
            
# likelyhood('deaths_discrepancy_percent')

# this = 'deaths_discrepancy_percent'

# dtype('deaths_discrepancy_percent')


In [45]:
for_max


{'white': {'population': 23405251,
  'ccvi': 42.51738863434971,
  'chance_of_infection': 7.014188043969955,
  'chance_of_death': 3.034227905530249},
 'black': {'population': 2253401,
  'ccvi': 29.466682281978308,
  'chance_of_infection': 14.831052639534542,
  'chance_of_death': 2.9430530468462153},
 'native': {'population': 302002,
  'ccvi': 33.278811969880664,
  'chance_of_infection': 8.975003700297991,
  'chance_of_death': 2.038781163434903},
 'asian': {'population': 5684633,
  'ccvi': 42.31817723757694,
  'chance_of_infection': 9.856392777186318,
  'chance_of_death': 3.272370579217633},
 'pacific': {'population': 154637,
  'ccvi': 36.37089456391097,
  'chance_of_infection': 29.67806658379855,
  'chance_of_death': 2.1071919377004122},
 'hispanic': {'population': 15288340,
  'ccvi': 26.486808674372583,
  'chance_of_infection': 29.645296255212646,
  'chance_of_death': 1.6169870446218715}}

In [11]:
meow = demographics.iloc[5:6]

values = meow.values[0]

meow.values.sum()



check = []

for i in range(1, len(races)):
    if races[i] != 'total':
        for_max[races[i]] = {}
        for_max[races[i]]['infection'] = (values[i-1]/values.sum())*100
        check.append((values[i-1]/values.sum())*100)

    
for_max

sum(check)


100.0

In [ ]:
# url = f'https://api.census.gov/data/2019/acs/acs5?get=NAME&for=place:*&in=state:{state_num}'

# response = requests.get(url).json()
# # print(json.dumps(response, indent=4, sort_keys=True))

# for r in response:
#     if r[0] == full_city:
#         url = f'https://api.census.gov/data/2019/acs/acs5?get=NAME&for=county%20(or%20part):*&in=state:{state_num}%20place:{r[2]}'
#         break


# response = requests.get(url).json()

# for r in response:
#     if r[0] != 'NAME': 
#         county_nums.append(r[3])

# for n in range(0, len(county_nums)):
#     if n == 0:
#         county_query = county_nums[n]
#     else:
#         county_query = county_query + f',{county_nums[n]}'


        
# url = f'https://api.census.gov/data/2019/acs/acs5?get=NAME,{pops}&for=tract:*&in=county:{county_query}&in=state:{state_num}'

# response = requests.get(url).json()

# for r in response:
#     if r[0] != 'NAME':
#         tracts.append({
#               'FIPS': int(f'{r[7]}{r[8]}{r[9]}'),
#               'white': int(r[1]),
#               'black': int(r[2]),
#               'native': int(r[3]),
#               'asian': int(r[4]),
#               'pacific': int(r[5]),
#               'hispanic': int(r[6])
#         })




In [ ]:
populations = pd.DataFrame(tracts)
populations

ccvi_and_pop = pd.merge(populations, ccvi, on='FIPS')

races = ['white','black','native','asian', 'pacific', 'hispanic']

for race in races:
    demogs[race]['population'] = ccvi_and_pop[race].sum()
    demogs[race]['ccvi'] = (ccvi_and_pop[race]*ccvi_and_pop['ccvi']).sum()/demogs[race]['population']

demogs


In [ ]:
# for n in range(0, len(county_nums)):
#     if n == 0:
#         county_query = state_num + county_nums[n]
#     else:
#         county_query = county_query + f',{state_num}{county_nums[n]}'

# county_query

In [ ]:
# ccvi['FIPS'] = ccvi['FIPS'].apply(lambda x: str(x)[:5])

# #!
# ccvi = pd.DataFrame(ccvi.groupby('FIPS')['ccvi'].mean())
# #!

# ccvi = ccvi.reset_index()
# ccvi = ccvi.rename(columns={'FIPS': 'county_fips'})

# ccvi

In [ ]:
# # drop the null rows
# ccvi = ccvi.dropna()
# ccvi.set_index('FIPS', inplace=True)
# ccvi

# ccvi.to_csv(r'../resources/ccvi_nonulls.csv')

In [ ]:
for n in range(0, len(county_nums)):
    county_nums[n] = state_num + county_nums[n]

county_nums

In [ ]:
cases = []


for n in range(0, len(county_nums)):

    url = f'https://data.cdc.gov/resource/n8mc-b4w4.json?$limit=300000&$offset={offset}&county_fips_code={county_nums[n]}'
    response = requests.get(url).json()


    cases.append(pd.DataFrame(response))

    limit = len(response)
    offset = offset + 1
    print(limit)
    print(url)
        
    

cases = pd.concat(cases).reset_index(drop=True)


# # Pretty print JSON for all launchpads
# response = requests.get(url).json()
# print(json.dumps(response, indent=4, sort_keys=True))

# cases = pd.DataFrame(response)
cases

#*
# cases = cases.loc[cases['county_fips_code'] != 'NA', :]
#*


In [ ]:
cases = []


for n in range(0, len(county_nums)):
    
    for race in races:

        url = f'https://data.cdc.gov/resource/n8mc-b4w4.json?$limit=300000&$offset={offset}&county_fips_code={county_nums[n]}'
        response = requests.get(url).json()


        cases.append(pd.DataFrame(response))

        limit = len(response)
        offset = offset + 1
        print(limit)
        print(url)
        
    

cases = pd.concat(cases).reset_index(drop=True)


# # Pretty print JSON for all launchpads
# response = requests.get(url).json()
# print(json.dumps(response, indent=4, sort_keys=True))

# cases = pd.DataFrame(response)
cases

#*
# cases = cases.loc[cases['county_fips_code'] != 'NA', :]
#*


In [ ]:
url = f'https://data.cdc.gov/resource/n8mc-b4w4.json?$limit=300000&$offset=0&race=White&county_fips_code={county_nums[0]}'
response = requests.get(url).json()
# print(json.dumps(response, indent=4, sort_keys=True))
pd.DataFrame(response)

In [ ]:
len(cases.loc[cases['res_county'] == 'JACKSON',:])

In [ ]:
cases['race'].unique()


In [ ]:
cases['ethnicity'].unique()

In [ ]:
case_counts = pd.DataFrame(cases['county_fips_code'].value_counts())

case_counts = case_counts.reset_index()
case_counts = case_counts.rename(columns={'index': 'county_fips', 'county_fips_code': 'case_count'})
case_counts = pd.merge(case_counts, ccvi, on='county_fips')

case_counts



In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model

In [ ]:
X = case_counts['ccvi'].values.reshape(-1, 1)
y = case_counts['case_count'].values.reshape(-1, 1)

# import necessary dependencies
from sklearn.model_selection import train_test_split

# create split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [ ]:
plt.scatter(X, y)

In [ ]:
model.fit(X_train, y_train)

model.score(X_test, y_test)


In [ ]:
model.predict()

# Quantifying Linear Regression

Create a model to quantify

In [ ]:
# Import dependencies
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression

# Generate some data
X, y = make_regression(n_samples=20, n_features=1, random_state=0, noise=4, bias=100.0)

# Create a linear model
model = LinearRegression()

# Fit (Train) our model to the data
model.fit(X, y)

## Quantifying our Model

* Mean Squared Error (MSE)

* R2 Score

There are a variety of ways to quantify the model, but MSE and R2 are very common

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Use our model to predict a value
predicted = model.predict(X)

# Score the prediction with mse and r2
mse = mean_squared_error(y, predicted)
r2 = r2_score(y, predicted)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")

A "good" MSE score will be close to zero while a "good" [R2 Score](https://en.wikipedia.org/wiki/Coefficient_of_determination) will be close to 1.

R2 Score is the default scoring for many of the Sklearn models

In [ ]:
# Overall Score for the model
model.score(X, y)

## Validation

We also want to understand how well our model performs on new data. 

One approach for this is to split your data into a training and testing dataset.

You fit (train) the model using training data, and score and validate your model using the testing data.

This train/test splitting is so common that Sklearn provides a mechanism for doing this. 

## Testing and Training Data

In order to quantify our model against new input values, we often split the data into training and testing data. The model is then fit to the training data and scored by the test data. Sklean pre-processing provides a library for automatically splitting up the data into training and testing

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Train the model using the training data

In [ ]:
model.fit(X_train, y_train)

And score the model using the unseen testing data

In [ ]:
model.score(X_test, y_test)

## Your Turn!